#코스 리뉴얼 : 중복제거 + 테마값 다시 업데이트 + 최적화 경로 + 위치값 업데이트 + 재 클러스터링

In [1]:
from tqdm import tqdm
import pandas as pd
import numpy as np
import psycopg2
from sklearn.cluster import KMeans
import json

In [13]:
course = pd.read_pickle("/content/drive/MyDrive/yeoreodigm/data_files/08.14_course.pickle")

In [5]:
course.head(2)

,id,title,places,nature,outdoor,fatigue,sea,walking,exciting,day,culture,cluster,length,east,west,south,north,main_location,optimize,optimized_route
0,1,[청주출발]연합패키지 제주를 반하다 3일_관광호텔 화순곶자왈+서커스+석예원한방족욕+...,"[602, 10, 58, 29]",1.250000,2.000000,1.250000,1.250000,2.750000,1.000000,1,0.250000,3,4,2.0,1.0,0.0,1.0,Mixed,id latitude longitude cluster 0 1...,"[[10, 29, 58, 602]]"
1,2,[청주출발]연합패키지 노팁/노옵션 제주 3일_관광호텔 산방산유람선+ 화순곶자왈+서커...,"[19, 37, 10, 602, 60, 58, 29]",1.714286,2.428571,1.285714,1.428571,2.571429,0.571429,2,0.428571,3,7,2.0,2.0,0.0,3.0,Mixed,id latitude longitude cluster 0 1...,"[[29, 58], [10, 19, 37, 60, 602]]"


In [4]:
def load_db():
  with open("/content/drive/MyDrive/yeoreodigm/data_files/db_info.json") as json_file:
    DB_INFO = json.load(json_file)
  
  endpoint = DB_INFO["ENDPOINT"]
  dbname = DB_INFO["DB_NAME"]
  user = DB_INFO["USER_ID"]
  password = DB_INFO["PASSWORD"]
  db = psycopg2.connect(host=endpoint,dbname=dbname,user=user,password=password)
  return db

In [6]:
attraction = pd.read_csv("/content/drive/MyDrive/yeoreodigm/data_files/07.31_attraction_db.csv",index_col=0)
attraction.head(2)

,id,title,address,introduction,latitude,longitude,dial_num,type,region1,region2,...,nature,outdoor,fatigue,sea,walking,exciting,day,culture,group,tag
0,1,사려니숲길,제주특별자치도 제주시 조천읍 교래리 산 137-1,"제주 숨은 비경 31, 삼나무 향기에 취하며 걷는 아름답고 청정한 숲길",33.40845,126.63976,064-900-8800,관광지,제주시,조천,...,2,2,1,1,3,1,0,0,8,"자연경관,커플,봄,흐림,걷기/등산,도보,친구"
1,2,우도(해양도립공원),제주특별자치도 제주시 우도면 삼양고수물길 1,소가 누워있는 형상을 하고 있는 제주의 가장 큰 부속섬,33.51949,126.95109,064-728-1527,관광지,섬 속의 섬,우도,...,3,3,3,1,3,0,3,0,7,"자연경관,맑음,아이,포토스팟"


In [14]:
course['places'] = course['places'].apply(lambda x: list(map(int,(x[1:-1]).split(','))))

In [17]:
for i in course:
  b= i
  break
b

'id'

In [18]:
course.shape

(3743, 20)

In [ ]:
course['ㄴ']

In [26]:
course['schedule'] = course['day']

###중복값 제거

In [86]:
course['places'] = course['places'].apply(lambda x: list(set(x)))

In [87]:
course.iloc[2000]

id                                                              2035
title                                               Jeju with 엄마.현지♡
places                                             [67, 61, 22, 295]
nature                                                           1.4
outdoor                                                          1.8
fatigue                                                          1.0
sea                                                              1.0
walking                                                          1.8
exciting                                                         1.0
day                                                              2.6
culture                                                          1.2
cluster                                                            2
length                                                             5
east                                                             0.0
west                              

##코스 테마값 업데이트

In [88]:
for i in tqdm(range(3743)):
  crs  = course.iloc[i]
  num_of_places = len(crs['places'])
  # print('len : ', num_of_places)
  # print(crs['day'])
  #length = crs['day'].values[0]
  nature_score = 0
  outdoor_score = 0
  fatigue_score = 0
  sea_score = 0
  walking_score = 0
  exciting_score = 0
  day_score = 0
  culture_score = 0
  for loc in crs['places']:
    now_loc = attraction.loc[attraction['id']==loc,['nature','outdoor','fatigue','sea','walking','exciting','day','culture']]
    #print(now_loc['nature'].values[0])
    nature_score += now_loc['nature'].values[0]
    outdoor_score += now_loc['outdoor'].values[0]
    fatigue_score += now_loc['fatigue'].values[0]
    sea_score += now_loc['sea'].values[0]
    walking_score += now_loc['walking'].values[0]
    exciting_score += now_loc['exciting'].values[0]
    day_score += now_loc['day'].values[0]
    culture_score += now_loc['culture'].values[0]
  
  course.iloc[i,3] = nature_score/num_of_places
  course.iloc[i,4] = outdoor_score/num_of_places
  course.iloc[i,5] = fatigue_score/num_of_places
  course.iloc[i,6] = sea_score/num_of_places
  course.iloc[i,7] = walking_score/num_of_places
  course.iloc[i,8] = exciting_score/num_of_places
  course.iloc[i,9] = day_score/num_of_places
  course.iloc[i,10] = culture_score/num_of_places

  


100%|██████████| 3743/3743 [00:54<00:00, 68.29it/s]


In [89]:
course.iloc[2000]

id                                                              2035
title                                               Jeju with 엄마.현지♡
places                                             [67, 61, 22, 295]
nature                                                          1.75
outdoor                                                         1.75
fatigue                                                          1.0
sea                                                              1.0
walking                                                         1.75
exciting                                                         1.0
day                                                              2.5
culture                                                          1.0
cluster                                                            2
length                                                             5
east                                                             0.0
west                              

###테마값 기반 클러스터링 진행 후 cluster에 삽입

In [41]:
from sklearn.cluster import KMeans

In [90]:
kmeans = KMeans(n_clusters=12,init='k-means++',random_state=0)
tags = ['nature','outdoor','fatigue','sea','walking','exciting','day','culture']

kmeans = kmeans.fit(course[tags])
course['cluster'] = kmeans.labels_

In [92]:
course.sample(1)

,id,title,places,nature,outdoor,fatigue,sea,walking,exciting,day,...,cluster,length,east,west,south,north,main_location,optimize,optimized_route,schedule
3130,3131,제주,"[2, 99, 8, 10, 13, 174, 46, 23, 310, 119, 25, ...",2.142857,2.857143,1.642857,1.928571,2.142857,1.571429,2.857143,...,6,16,6.0,5.0,2.0,3.0,Mixed,id latitude longitude cluster 0 ...,"[[2, 310], [8, 10, 99, 119, 174, 223], [13, 25...",4


In [93]:
course.groupby('cluster').mean()

,id,nature,outdoor,fatigue,sea,walking,exciting,day,culture,length,east,west,south,north,schedule
cluster,,,,,,,,,,,,,,,
0,1807.695122,0.748391,0.969560,1.031198,1.138434,1.118332,1.075545,2.447668,1.413842,5.560976,0.951220,2.036585,1.109756,1.463415,1.512195
1,1919.065672,2.056554,2.366501,1.216282,1.427500,1.606170,0.901688,2.135033,0.368872,9.017910,1.576119,2.740299,2.352239,2.349254,2.602985
2,1992.539846,2.439533,2.871695,1.923374,1.514465,2.442265,0.779578,2.766817,0.257057,9.033419,4.200514,1.876607,1.187661,1.768638,2.552699
3,1898.206061,1.671573,2.131577,1.422555,1.393249,1.829580,1.126366,2.547384,0.541217,9.900000,2.866667,2.948485,1.787879,2.296970,2.851515
4,1808.045977,1.525114,2.088999,1.352843,1.482451,1.264163,1.657808,2.570403,0.321276,7.206897,1.505747,2.304598,1.471264,1.925287,1.971264
5,1926.388889,1.351278,1.637525,1.134069,1.218824,1.501164,0.991022,2.275096,0.824809,7.750000,1.250000,2.932540,1.730159,1.837302,2.146825
6,1724.872131,2.240853,2.804511,1.579502,2.070795,2.130141,1.202102,2.677704,0.185692,8.209836,2.908197,2.075410,1.049180,2.177049,2.298361
7,1881.398955,2.363282,2.695972,1.634290,1.307867,2.048907,0.629547,2.500557,0.409168,11.857143,3.860627,2.813589,2.693380,2.489547,3.465157
8,1991.355814,2.153232,2.636317,1.624000,1.409133,1.811780,1.207833,2.612982,0.255593,10.413953,3.055814,2.741860,2.430233,2.186047,2.983721


##위치정보 반영

In [55]:
east = ['구좌','성산','표선','우도']
west = ['안덕','대정','한경','한림','추자도','가파도','마라도','비양도','차귀도']
north = ['제주시내','애월','조천']
south = ['서귀포시내','남원','중문']

In [56]:
location_template = [east,west,south,north]

In [57]:
attraction.sample()

,id,title,address,introduction,latitude,longitude,dial_num,type,region1,region2,...,nature,outdoor,fatigue,sea,walking,exciting,day,culture,group,tag
920,921,목시물굴,제주특별자치도 제주시 조천읍 선흘리 산26번지 일대,선흘리 초토화 이후 선흘리민들이 은신 했다가 희생 당한 곳,33.51025,126.722336,--,관광지,제주시,조천,...,3,3,1,2,1,1,3,0,3,문화관광


In [94]:
entire_direction_point = []
for crs in tqdm(course['places']):
  direction_point = np.zeros(4)
  for location in crs:
    region = attraction.loc[attraction['id']==location,'region2'].values[0]
    #region_list.append(region)
    for i in range(4):
      if region in location_template[i]:
        direction_point[i] += 1
  entire_direction_point.append(direction_point)  

100%|██████████| 3743/3743 [00:09<00:00, 380.86it/s]


In [95]:
direction_df = pd.DataFrame(entire_direction_point)
direction_df.columns = ['east','west','south','north']

In [96]:
course.shape

(3743, 21)

In [97]:
direction_df

,east,west,south,north
0,2.0,1.0,0.0,1.0
1,2.0,2.0,0.0,3.0
2,2.0,1.0,0.0,2.0
3,1.0,0.0,0.0,2.0
4,2.0,1.0,0.0,2.0
...,...,...,...,...
3738,4.0,4.0,2.0,1.0
3739,5.0,3.0,6.0,4.0
3740,2.0,1.0,5.0,1.0
3741,0.0,3.0,2.0,2.0


###실제 값을 이용해서 확인

In [98]:
direction_df.iloc[2000]

east     0.0
west     3.0
south    1.0
north    0.0
Name: 2000, dtype: float64

In [99]:
course.iloc[2000]

id                                                              2035
title                                               Jeju with 엄마.현지♡
places                                             [67, 61, 22, 295]
nature                                                          1.75
outdoor                                                         1.75
fatigue                                                          1.0
sea                                                              1.0
walking                                                         1.75
exciting                                                         1.0
day                                                              2.5
culture                                                          1.0
cluster                                                            5
length                                                             5
east                                                             0.0
west                              

In [119]:
course[['east','west','south','north']] = direction_df[['east','west','south','north']].values

In [120]:
course.iloc[2000]

id                                                              2035
title                                               Jeju with 엄마.현지♡
places                                             [67, 61, 22, 295]
nature                                                          1.75
outdoor                                                         1.75
fatigue                                                          1.0
sea                                                              1.0
walking                                                         1.75
exciting                                                         1.0
day                                                              2.5
culture                                                          1.0
cluster                                                            5
length                                                             5
east                                                             0.0
west                              

In [121]:
course.sample(2)

,id,title,places,nature,outdoor,fatigue,sea,walking,exciting,day,...,cluster,length,east,west,south,north,main_location,optimize,optimized_route,schedule
1627,1628,쭈도?,"[37, 8, 9, 10, 15, 16, 20, 223]",2.625,2.875000,1.625000,1.750000,1.750000,1.000000,2.375000,...,8,8,1.0,2.0,3.0,2.0,Mixed,id latitude longitude cluster 0 ...,"[[8, 10, 15, 16, 20, 37, 223], [9]]",2
1251,1251,제주에 푹 담궈진 2박3일,"[3, 9, 10, 13, 110, 270, 16, 18, 153, 92, 25]",2.000,2.363636,1.454545,1.454545,2.363636,0.818182,2.727273,...,11,12,4.0,2.0,4.0,1.0,Mixed,id latitude longitude cluster 0 ...,"[[10, 16, 110, 153], [18, 92, 270], [3, 9, 13,...",3


In [122]:
save_max_location = []
for crs in entire_direction_point:
  max_idx = crs.argmax()
  max_ratio = crs[max_idx] / crs.sum()
  if max_ratio > 0.5:
    if max_idx == 0:
      save_max_location.append("East")
    elif max_idx==1:
      save_max_location.append("West")
    elif max_idx==2:
      save_max_location.append("South")
    else:
      save_max_location.append("North")
  else:
    save_max_location.append("Mixed")
  #max_ratio = f"{max_ratio:.2f}"
  #print(crs, max_ratio,max_idx,save_max_location[cnt])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: RuntimeWarning: invalid value encountered in double_scalars
  after removing the cwd from sys.path.


In [123]:
course['main_location'] = save_max_location

In [124]:
course.sample(3)

,id,title,places,nature,outdoor,fatigue,sea,walking,exciting,day,...,cluster,length,east,west,south,north,main_location,optimize,optimized_route,schedule
3148,3149,제주도,"[1, 223, 3, 229, 8, 13, 46, 270, 52, 22, 24, 2...",2.384615,2.769231,1.384615,2.000000,2.538462,1.076923,2.538462,...,6,14,5.0,3.0,1.0,4.0,Mixed,id latitude longitude cluster 0 ...,"[[3, 13, 52, 229], [31, 223], [1, 25, 46, 270]...",4
1196,1196,3.5박 5일 제주도,"[3, 2, 59]",2.666667,3.000000,2.666667,1.000000,2.666667,0.333333,2.666667,...,9,3,2.0,0.0,0.0,1.0,East,id latitude longitude cluster 0 2 ...,"[[2, 3, 59]]",1
3281,3282,제주도,"[4, 6, 8, 174, 47, 83, 30]",1.428571,2.142857,1.428571,1.285714,2.000000,1.142857,2.714286,...,3,7,3.0,2.0,0.0,2.0,Mixed,id latitude longitude cluster 0 ...,"[[4, 6, 30, 47, 83], [8, 174]]",2


In [125]:
course.iloc[2000]

id                                                              2035
title                                               Jeju with 엄마.현지♡
places                                             [67, 61, 22, 295]
nature                                                          1.75
outdoor                                                         1.75
fatigue                                                          1.0
sea                                                              1.0
walking                                                         1.75
exciting                                                         1.0
day                                                              2.5
culture                                                          1.0
cluster                                                            5
length                                                             5
east                                                             0.0
west                              

In [126]:
course['length'] = course['places'].apply(lambda x: len(x))

In [127]:
course.iloc[2000]

id                                                              2035
title                                               Jeju with 엄마.현지♡
places                                             [67, 61, 22, 295]
nature                                                          1.75
outdoor                                                         1.75
fatigue                                                          1.0
sea                                                              1.0
walking                                                         1.75
exciting                                                         1.0
day                                                              2.5
culture                                                          1.0
cluster                                                            5
length                                                             4
east                                                             0.0
west                              

##새로 업데이트한 코스(중복제거 + 테마값 다시 업데이트 + 최적화 경로 + 위치값 업데이트 + 재 클러스터링) 저장하기

In [128]:
course.to_pickle("/content/drive/MyDrive/yeoreodigm/data_files/08.15_course.pickle")

##리뉴얼 코스 클러스터별로 이름 붙이기

0 : 쉿! 같이 걷자 제주
1 : 제주에선, 잠시 쉬어가도 괜찮아.
2 : 실내에서 즐기는 제주 인문학 탐방
3 : 20대라면, 사진 찍기 좋은 제주 감성여행
4 : 부모님과 함께라면, 더 좋은 제주
5 : 맑고 푸르른 제주의 쪽빛 바다
6 : 몸으로 체험하는 제주의 자연
7 : 화산이 빚어낸 제주의 고요한 녹빛
8 : 시간이 빚어낸 제주의 푸르른 경관
9 : 실내외를 아우르는 제주여행
10 : 제주, 다양하게 즐기기
11 : 신나게 즐겨보는 액티브 인 제주

In [132]:
course.groupby('cluster').mean()[['nature','outdoor','fatigue','sea','walking','exciting','day','culture']].sort_values(by='nature',ascending=True)

,nature,outdoor,fatigue,sea,walking,exciting,day,culture
cluster,,,,,,,,
0,0.748391,0.969560,1.031198,1.138434,1.118332,1.075545,2.447668,1.413842
5,1.351278,1.637525,1.134069,1.218824,1.501164,0.991022,2.275096,0.824809
4,1.525114,2.088999,1.352843,1.482451,1.264163,1.657808,2.570403,0.321276
3,1.671573,2.131577,1.422555,1.393249,1.829580,1.126366,2.547384,0.541217
10,1.832400,2.226441,1.544675,1.115327,1.915520,0.593481,2.542839,0.942889
11,1.966507,2.430212,1.462352,1.423444,2.304504,0.873817,2.420382,0.237495
1,2.056554,2.366501,1.216282,1.427500,1.606170,0.901688,2.135033,0.368872
8,2.153232,2.636317,1.624000,1.409133,1.811780,1.207833,2.612982,0.255593
6,2.240853,2.804511,1.579502,2.070795,2.130141,1.202102,2.677704,0.185692


0 : 실내에서 즐기는 제주 인문학 탐방
5 : 실내외를 아우르는 제주여행
4 : 신나게 즐겨보는 액티브 인 제주
3 : 제주, 다양하게 즐기기
10 : 20대라면, 사진 찍기 좋은 제주 감성여행
11: 쉿! 같이 걷자 제주
1 : 제주에선, 잠시 쉬어가도 괜찮아.
8 : 몸으로 체험하는 제주의 자연
6 : 맑고 푸르른 제주의 쪽빛 바다
7 : 부모님과 함께라면, 더 좋은 제주
2 : 시간이 빚어낸 제주의 푸르른 경관
9 : 화산이 빚어낸 제주의 고요한 녹빛



###클러스터별로 지정한 title dictionary로 만들기

In [146]:
cluster_dict = {0 : "실내에서 즐기는 제주 인문학 탐방",\
                5 : "실내외를 아우르는 제주여행",\
                4 : "신나게 즐겨보는 액티브 인 제주" ,\
                3 : "제주, 다양하게 즐기기",\
                10 : "20대라면, 사진 찍기 좋은 제주 감성여행",\
                11: "쉿! 같이 걷자 제주",\
                1 : "제주에선, 잠시 쉬어가도 괜찮아.",\
                8 : "직접 체험하며 즐기는 제주",\
                6 : "맑고 푸르른 제주의 쪽빛 바다",\
                7 : "부모님과 함께라면, 더 좋은 제주",\
                2 : "시간이 빚어낸 제주의 푸르른 경관",\
                9 : "화산이 빚어낸 제주의 고요한 녹빛",\
                }

In [137]:
cluster_dict[4]

'신나게 즐겨보는 액티브 인 제주'

In [ ]:
ㅊ

In [133]:
course.head(1)

,id,title,places,nature,outdoor,fatigue,sea,walking,exciting,day,...,cluster,length,east,west,south,north,main_location,optimize,optimized_route,schedule
0,1,[청주출발]연합패키지 제주를 반하다 3일_관광호텔 화순곶자왈+서커스+석예원한방족욕+...,"[602, 10, 58, 29]",1.25,2.0,1.25,1.25,2.75,1.0,1.75,...,11,4,2.0,1.0,0.0,1.0,Mixed,id latitude longitude cluster 0 1...,"[[10, 29, 58, 602]]",1


In [134]:
course['title_old'] = course['title']

In [138]:
def change_title(cluster):
  return cluster_dict[cluster]

In [139]:
change_title(4)

'신나게 즐겨보는 액티브 인 제주'

In [147]:
course['title'] = course['cluster'].apply(lambda x: change_title(x))

In [148]:
course.sample(20)[['title','cluster']]

,title,cluster
2238,쉿! 같이 걷자 제주,11
1690,"부모님과 함께라면, 더 좋은 제주",7
975,"20대라면, 사진 찍기 좋은 제주 감성여행",10
441,신나게 즐겨보는 액티브 인 제주,4
1331,쉿! 같이 걷자 제주,11
1054,"부모님과 함께라면, 더 좋은 제주",7
2573,"제주, 다양하게 즐기기",3
2887,직접 체험하며 즐기는 제주,8
3422,"제주, 다양하게 즐기기",3
3435,"20대라면, 사진 찍기 좋은 제주 감성여행",10


###중복 데이터를 제거했으므로 길이가 3미만인 코스는 제거

In [156]:
course.loc[course['length']<3]

,id,title,places,nature,outdoor,fatigue,sea,walking,exciting,day,...,length,east,west,south,north,main_location,optimize,optimized_route,schedule,title_old
140,141,맑고 푸르른 제주의 쪽빛 바다,"[8, 187]",1.0,3.0,1.5,2.5,1.5,1.0,2.0,...,2,0.0,2.0,0.0,0.0,West,id latitude longitude cluster 0 ...,[],0,시험기간 중간에 떠나는 힐링 2박 3일 제주여행
155,156,시간이 빚어낸 제주의 푸르른 경관,"[8, 279]",2.5,3.0,2.5,1.5,2.5,1.0,3.0,...,2,0.0,2.0,0.0,0.0,West,id latitude longitude cluster 0 ...,[],0,한적한 곳만 골라가는 나혼자 제주도
342,343,화산이 빚어낸 제주의 고요한 녹빛,[94],3.0,3.0,3.0,1.0,3.0,1.0,3.0,...,1,0.0,1.0,0.0,0.0,West,id latitude longitude cluster 0 94 ...,[],0,비양도의 모든 명소를 소개한다!
448,449,신나게 즐겨보는 액티브 인 제주,"[202, 155]",1.5,3.0,2.0,1.0,1.0,3.0,3.0,...,2,0.0,1.0,0.0,1.0,Mixed,id latitude longitude cluster 0 15...,[],0,"승마, 사냥과 함께하는 제주 여행"
697,698,쉿! 같이 걷자 제주,"[592, 204]",2.5,2.5,1.0,1.5,2.5,1.0,1.5,...,2,0.0,0.0,0.0,0.0,Mixed,id latitude longitude cluster 0 20...,[],0,추자도 1박 2일 낭만여행
906,907,시간이 빚어낸 제주의 푸르른 경관,"[8, 28]",2.5,3.0,2.0,1.5,2.5,1.0,2.5,...,2,0.0,2.0,0.0,0.0,West,id latitude longitude 0 8 33.39374...,[],0,느영나영 제주여행 season 2
1126,1127,신나게 즐겨보는 액티브 인 제주,"[202, 155]",1.5,3.0,2.0,1.0,1.0,3.0,3.0,...,2,0.0,1.0,0.0,1.0,Mixed,id latitude longitude 0 155 33.354...,[],0,친구와 승마하러 제주도에 가다 1박2일
1702,1703,맑고 푸르른 제주의 쪽빛 바다,"[13, 382]",2.5,3.0,1.5,3.0,1.5,2.0,3.0,...,2,2.0,0.0,0.0,0.0,East,id latitude longitude 0 13 33.555...,[],0,두번 째 제주도 여행
1790,1791,쉿! 같이 걷자 제주,"[338, 10]",1.5,2.5,1.0,1.5,3.0,0.5,1.0,...,2,0.0,1.0,1.0,0.0,Mixed,id latitude longitude 0 10 33.290...,[],0,24시간 chrismas jeju!
1793,1794,"20대라면, 사진 찍기 좋은 제주 감성여행","[211, 4]",1.0,3.0,2.0,1.5,3.0,0.5,3.0,...,2,2.0,0.0,0.0,0.0,East,id latitude longitude 0 4 33.491...,[],0,제주여행


In [157]:
course.shape

(3743, 22)

In [158]:
course = course[course['length']>2]

In [159]:
course.shape

(3725, 22)

###file로 저장

In [160]:
course.to_pickle("/content/drive/MyDrive/yeoreodigm/data_files/08.15_course.pickle")

##DB에 전부 반영

In [151]:
db = load_db()
cursor =db.cursor()

In [155]:
course.sample()

,id,title,places,nature,outdoor,fatigue,sea,walking,exciting,day,...,length,east,west,south,north,main_location,optimize,optimized_route,schedule,title_old
1341,1341,실내외를 아우르는 제주여행,"[8, 10, 75, 108, 22, 61]",1.0,1.333333,1.0,1.333333,1.666667,1.0,2.333333,...,6,0.0,5.0,1.0,0.0,West,id latitude longitude cluster 0 ...,"[[8, 10, 22, 75, 108], [61]]",2,2박 3일 제주도여행


###table[course_] 에 먼저 올리기


In [161]:
for crs in tqdm(course.values):
  cursor.execute("insert into course_(title,places,nature,outdoor,fatigue,sea,walking,exciting,day,culture,cluster,length,east,west,south,north,main_location) VALUES(%s, %s, %s, %s, %s,%s, %s, %s,%s, %s, %s,%s, %s, %s,%s, %s, %s)",(crs[1],crs[2],crs[3],crs[4],crs[5],crs[6],crs[7],crs[8],crs[9],crs[10],crs[11],crs[12],crs[13],crs[14],crs[15],crs[16],crs[17]))
db.commit()

100%|██████████| 3725/3725 [07:30<00:00,  8.26it/s]


###travel_note에 올리기

In [163]:
import datetime

In [164]:
a = datetime.date(2020, 1, 11)
a

datetime.date(2020, 1, 11)

In [165]:
a+datetime.timedelta(4)

datetime.date(2020, 1, 15)

In [169]:
tmp23 = course.iloc[1324]
tmp23['schedule']

2

In [170]:
len(course)

3725

In [171]:
import random

In [172]:
attraction.sample()

,id,title,address,introduction,latitude,longitude,dial_num,type,region1,region2,...,nature,outdoor,fatigue,sea,walking,exciting,day,culture,group,tag
543,544,두산봉(말미오름),제주특별자치도 서귀포시 성산읍 시흥리 산 1-5,올레1코스의 마스코트 두산봉,33.479509,126.884147,NaN,관광지,서귀포시,성산,...,3,3,1,0,1,0,2,1,3,"자연경관,커플,봄,도보,친구"


In [197]:
db.rollback()

In [198]:
for i in tqdm(range(len(course))):
  now = course.iloc[i]
  now_path = now['optimized_route']
  places = now['places']
  now_cluster = now['cluster']
  now_location_info = np.array(list(now[['east','west','south','north']]))
  #썸네일은 구성 목적지 중에서 random으로하나 뽑아서 그놈의 이미지 경로 쓰자
  thumbnail = attraction.loc[attraction['id'] == random.choice(places),'image_url'].values[0]
  max_idx = now_location_info.argmax()
  max_ratio = now_location_info[max_idx] / now_location_info.sum()
  region = []
  if max_ratio > 0.5:
    if max_idx == 0:
      region.append("제주 동부")
    elif max_idx==1:
      region.append("제주 서부")
    elif max_idx==2:
      region.append("제주 남부")
    else:
      region.append("제주 북부")
  else:
    if now_location_info[0] > 0 :
      region.append("제주 동부")
    if now_location_info[1] > 0 :
      region.append("제주 서부")
    if now_location_info[2] > 0 :
      region.append("제주 남부")
    if now_location_info[3] > 0 :
      region.append("제주 북부")



  # print('cluster : ',now_cluster  ,  "location : ",now_location_info)
  travel_note_id = i+1
  member_id = 1
  title = now['title']
  
  start_day = datetime.date(2022,8,14)
  end_day = start_day + datetime.timedelta(int(now['schedule'])-1) 
  # print("*******"*10)
  # print(start_day,end_day)
  # print(travel_note_id,1,title,start_day,end_day,region,True,thumbnail)


  #sql = "insert into travel_note(travel_note_id, member_id, title, day_start,day_end,adult,region,public_share,thumbnail) values (%s, %s, %s, %s, %s, %s, %s)",(travel_note_id,1,title,start_day,end_day,region,True,thumbnail)
  cursor.execute("insert into travel_note(travel_note_id, member_id, title, day_start,day_end,adult,places_input,region,theme,public_share,companion,thumbnail) values (%s, %s, %s, %s,%s, %s, %s,%s,%s, %s,%s,%s)",(travel_note_id,1,title,start_day,end_day,2,[],region,[],True,[],thumbnail))
  
  for idx,path in enumerate(now_path):
    #course_sql = "insert into course(travel_note_id, day, places) VALUES(%s, %s, %s )",(travel_note_id, idx+1, path)
    cursor.execute("insert into course(travel_note_id, day, places) VALUES(%s, %s, %s )",(travel_note_id, idx+1, path))


db.commit()

100%|██████████| 3725/3725 [27:43<00:00,  2.24it/s]
